In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
import os
import PIL

train_dir = "../input/train/"
labels = []
train_imgs = []
from PIL import Image

for filename in os.listdir(train_dir):
    label, _, _ = filename.strip().split('.')
    img = PIL.Image.open(train_dir + filename)
    I = np.asarray(img.resize([128, 128]))
    if label == 'cat':
        labels.append(0)
    else:
        labels.append(1)
    train_imgs.append(I)

# Any results you write to the current directory are saved as output.

In [ ]:
train_imgs = np.asarray(train_imgs)
labels = np.asarray(labels)
print(train_imgs.shape)
print(labels.shape)

In [ ]:
import tensorflow as tf

def cnn_model(imgs, is_training = True):
    
    
    conv1 = tf.layers.conv2d(imgs, 64, [5, 5], [2, 2], padding = 'same', activation = tf.nn.relu)
    bn1 = tf.layers.batch_normalization(conv1, axis = 1)
    conv2 = tf.layers.conv2d(bn1, 128, [5, 5], [2, 2], padding = 'same', activation = tf.nn.relu)
    bn2 = tf.layers.batch_normalization(conv2, axis = 1)
    flat = tf.reshape(bn2, [tf.shape(imgs)[0], 32*32*128])
    a1 = tf.layers.dense(flat, 1024, activation = tf.nn.relu)
    logtis = tf.layers.dense(a1, 2)
    return logtis
    


In [ ]:

with tf.Session() as sess:
    imgs = tf.placeholder(tf.float32, [None, 128, 128, 3])
    y = tf.placeholder(tf.int32, [None])
    logtis = cnn_model(imgs)
    onehot_labels = tf.one_hot(y, depth=2)
    loss = tf.losses.softmax_cross_entropy(onehot_labels, logtis)

    optimizer = tf.train.AdamOptimizer()
    optimizer.minimize(loss)

    pred = tf.argmax(logtis, axis = 1)
    print(type(labels))
    _loss = sess.run([loss], {imgs : train_imgs[0:100], y : labels[0:100]})
    print(_loss)